In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('combined-dataset/final_reviews_data.csv')

# Encode user_id and place_id
user_encoder = LabelEncoder()
place_encoder = LabelEncoder()

df['user_id_encoded'] = user_encoder.fit_transform(df['user_id'])
df['place_id_encoded'] = place_encoder.fit_transform(df['id'])

# Display the DataFrame with encoded columns
df.head()


,id,types,review_number,review,user_id,sentiment,user_id_encoded,place_id_encoded
0,ChIJYcGr7GSb0S0RckePBrCWikw,"hotel, lodging",review 1,"It has quite small room, and the hallway is qu...",user_18425,3.02,3703,3660
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"indonesian_restaurant, restaurant, food",review 1,"Surprisingly, a really good warung that’s hidd...",user_46108,3.42,15699,3762
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"tourist_attraction, restaurant, food",review 1,"Only had a fleeting visit here, came by coach,...",user_43063,2.97,14376,3678
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"coffee_shop, cafe, food, store",review 1,"One word, underrated! How come place like this...",user_18528,3.69,3748,829
4,ChIJxaITmQia0S0RyrbukE8vsJU,"tourist_attraction, place_of_worship",review 1,"This temple is located in Singaraja, located i...",user_19259,3.28,4073,6310


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Flatten, Dense, Input, Concatenate
from tensorflow.keras.models import Model

# Number of unique users and places
num_users = df['user_id_encoded'].nunique()
num_places = df['place_id_encoded'].nunique()

# Define the embedding size
embedding_size = 16

# User input
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(num_users, embedding_size, name='user_embedding')(user_input)
user_vec = Flatten(name='user_flatten')(user_embedding)

# Place input
place_input = Input(shape=(1,), name='place_input')
place_embedding = Embedding(num_places, embedding_size, name='place_embedding')(place_input)
place_vec = Flatten(name='place_flatten')(place_embedding)

# Concatenate user and place vectors
concat = Concatenate()([user_vec, place_vec])

# Add dense layers
dense = Dense(128, activation='relu')(concat)
output = Dense(1, activation='linear')(dense)

# Build the model
model = Model(inputs=[user_input, place_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Display the model summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ place_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 16)     │    411,072 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ place_embedding     │ (None, 1, 16)     │    104,768 │ place_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_flatten        │ (None, 16)        │          0 │ user_embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ place_flatten       │ (None, 16)        │          0 │ place_embedding[… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32)        │          0 │ user_flatten[0][… │
│ (Concatenate)       │                   │            │ place_flatten[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      4,224 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │        129 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 520,193 (1.98 MB)

 Trainable params: 520,193 (1.98 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
# Train the model
history = model.fit(
    [df['user_id_encoded'], df['place_id_encoded']],
    df['sentiment'],
    epochs=10,
    batch_size=1,
    verbose=1
)


Epoch 1/10
32745/32745 ━━━━━━━━━━━━━━━━━━━━ 74s 2ms/step - loss: 0.4839
Epoch 2/10
32745/32745 ━━━━━━━━━━━━━━━━━━━━ 73s 2ms/step - loss: 0.1944
Epoch 3/10
32745/32745 ━━━━━━━━━━━━━━━━━━━━ 84s 3ms/step - loss: 0.1319
Epoch 4/10
32745/32745 ━━━━━━━━━━━━━━━━━━━━ 96s 3ms/step - loss: 0.0975
Epoch 5/10
32745/32745 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - loss: 0.0784
Epoch 6/10
32745/32745 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - loss: 0.0633
Epoch 7/10
32745/32745 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - loss: 0.0515
Epoch 8/10
32745/32745 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - loss: 0.0424
Epoch 9/10
32745/32745 ━━━━━━━━━━━━━━━━━━━━ 77s 2ms/step - loss: 0.0357
Epoch 10/10
32745/32745 ━━━━━━━━━━━━━━━━━━━━ 63s 2ms/step - loss: 0.0306


In [4]:
# Evaluate the model
loss = model.evaluate([df['user_id_encoded'], df['place_id_encoded']], df['sentiment'])
print(f"Model Loss: {loss}")


1024/1024 ━━━━━━━━━━━━━━━━━━━━ 1s 872us/step - loss: 0.0245
Model Loss: 0.025122590363025665


In [7]:
import numpy as np

# Function to recommend places for a given user
def recommend_places(user_id, model, df, user_encoder, place_encoder, top_n=5):
    user_idx = user_encoder.transform([user_id])[0]
    all_place_ids = df['place_id_encoded'].unique()
    user_array = np.array([user_idx] * len(all_place_ids))
    
    predictions = model.predict([user_array, all_place_ids])
    
    place_indices = predictions.flatten().argsort()[-top_n:][::-1]
    recommended_places = place_encoder.inverse_transform(place_indices)
    
    return recommended_places

# Get recommendations for a specific user
user_id = "user_18425"
recommended_places = recommend_places(user_id, model, df, user_encoder, place_encoder)
print(f"Recommended places for user {user_id}: {recommended_places}")

places = pd.read_csv('combined-dataset/combined_datasetV2.csv')
places = places[places['id'].isin(recommended_places)]  # Filter places
places


205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Recommended places for user user_18425: ['ChIJUUIIkCVb0i0RjJ5iXpnEoaU' 'ChIJeWatXmhb0i0Rnn-13EHMqiY'
 'ChIJp3K9sSM50i0RLucGqMDEhPU' 'ChIJcQiarIwY0i0Rxq7rnU0QE80'
 'ChIJ22zYshtF0i0RENOIShDHnXo']


,id,name,latitude,longitude,address,url,status,phone,primary-type,types,rating,rating-count,price-level,review 1,review 2,review 3,review 4,review 5
12345,ChIJcQiarIwY0i0Rxq7rnU0QE80,Warung Umadue,-8.436345,115.307274,"Jl, Jl. Astinapura Sel., Banjar Kelodan, Kec. ...",https://maps.google.com/?cid=14777172727880724166,CLOSED_TEMPORARILY,+62 813-5397-6161,indonesian_restaurant,"indonesian_restaurant, restaurant, food",4.4,63.0,PRICE_LEVEL_INEXPENSIVE,Good quality local balibese food. So autentik....,a good clean restaurant with rice field view. ...,A beautifully built warung with taro tanah wal...,We had such a great time spent here. The food ...,Love the cozy little space yet very beautiful ...
27765,ChIJp3K9sSM50i0RLucGqMDEhPU,The Palms Canggu,-8.646618,115.136390,"Jl. Canggu Padang Linjong Gg. Agung Roy, Cangg...",https://maps.google.com/?cid=17691481567949940526,OPERATIONAL,+62 813-3806-7559,hotel,"hotel, lodging",4.6,115.0,NaN,"Clean and modern hotel, nice pool with seating...","Little hidden paradise, we walked through the ...",I have stayed at The Palms several times and I...,I have lived in Palms for 11 months from Febru...,"Enjoyed a lovely stay at this hotel, it felt l..."
31329,ChIJ22zYshtF0i0RENOIShDHnXo,Horison Le Aman Hotel,-8.762298,115.178592,"Jl. Bypass Ngurah Rai No.50, Jimbaran, Kec. Ku...",https://maps.google.com/?cid=8835436916754666256,OPERATIONAL,+62 361 4466000,hotel,"hotel, lodging",4.5,195.0,NaN,Excellent!\nFamily trip to Bali and got afford...,"My room was booked via an Airbnb, and it was t...",I stayed for one night in twin deluxe bedroom....,1000/10 IT WAS NICE TRIP WHEN I GO TO BALII YE...,"clean and comfortable hotel, strategic locatio..."
32763,ChIJUUIIkCVb0i0RjJ5iXpnEoaU,Adhiloka Uluwatu,-8.827930,115.149406,"Jl. Pura Masuka No.168, Ungasan, Kec. Kuta Sel...",https://maps.google.com/?cid=11935036650501086860,OPERATIONAL,NaN,hotel,"hotel, lodging",4.8,82.0,NaN,"Thank you to everyone, especially the boys at ...",Really good hotel for a really good price. I l...,I had a wonderful experience at Adhiloka last ...,One of my all time favourite places I’ve staye...,It was what we expected out of the property.\n...
33306,ChIJeWatXmhb0i0Rnn-13EHMqiY,Ocho Bali Villa,-8.837684,115.175184,"Jl. Alas Arum No.37, Kutuh, Kec. Kuta Sel., Ka...",https://maps.google.com/?cid=2786263902729895838,OPERATIONAL,+62 878-6201-1196,NaN,"resort_hotel, hotel, lodging",4.4,194.0,NaN,This hotel was so beautiful and small. If you ...,"good day, this place is amazing it has its own...",Stayed here 3 days 2 night\nLocated in Uluwatu...,Staff were very welcoming and allowed us to ch...,The room was perfect.\nBreakfast was good.\nIt...
